In [8]:
from library.plots import plot_heatmap, plot_surface
import pandas as pd
import torch as t
import time, datetime

# profile the code with cProfile
import cProfile
import pstats

from tqdm import tqdm
from copy import deepcopy

template = "plotly_dark"

In [2]:
def time_now():
    return datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")


# Edge

In [3]:
edges_slits = [[-50, 10]] # 1 slit
barrier_points = []
gravity_nodes_x = t.tensor([10])

# 1 Slit


In [17]:
edges_slits = [[-20, 20]]  # 1 slit
barrier_points = [[-35, 35]]
# barrier_points = [[-10, 10]]
gravity_nodes_x = t.tensor(edges_slits).reshape(1, -1).squeeze()



# 2 Slits


In [40]:

edges_slits = [[-25, -5], [5, 25]]  # two slits
barrier_points = [[-i, i] for i in range(26, 35)] + [[-i, i] for i in range(1, 4)]
gravity_nodes_x = t.tensor(edges_slits + barrier_points).reshape(1, -1).squeeze().sort()[0]


# Simulation

In [48]:

# start profiler
# pr = cProfile.Profile()
# pr.enable()

device = t.device("cuda")

n_slits = len(edges_slits)
n_particles = 10_000  # <3 3080Ti
barrier_y = 0
dt = 1
v_y = 3  # FIX
x = 50
y = 3000  # 10000
type_trig = "sin"
# G = -0.6626  # why is this negative?
G = 2
mass = 1  # FIX
wavelength = 0.1
power = 2
inner_power = 2
round_x = 5
n_bins = 100  # always odd numbers
PLOT_3D = True
PLOT_Z_DISTRIBUTION = False
PLOT_DENSITY = False
normalize = True
moving_wave = False
gravity_threshold = 500

p_x = t.Tensor()

for slit in edges_slits:
    p_x = t.cat((p_x, t.linspace(slit[0], slit[1], n_particles + 1)), 0)
    # remove the particles that would be superpositioned with edges
    # p_x = p_x[1:-1]
p_y = t.zeros_like(p_x)
p_xy = t.stack((p_x, p_y), 1).to(device)


In [49]:

# Let's try to keep the trail on the cpu
trail_xy = t.clone(p_xy.unsqueeze(0))
v_xy = t.zeros_like(p_xy).to(device) + t.tensor((0, v_y)).to(device)

p_mass = t.ones_like(p_y).to(device)
o_y = t.zeros_like(gravity_nodes_x) + barrier_y
o_xy = t.stack((gravity_nodes_x, o_y), 1).to(device)

# Stupid work around, code is becoming cluttered
dist = t.Tensor().to(device)
hist = t.histc(trail_xy[0][:, 0].to("cpu"), bins=n_bins, min=-x, max=x)
dist = t.cat((dist, hist.unsqueeze(0).to(device)), 0)

t_ = 0
for i in tqdm(range(y)):
    if i <= gravity_threshold:
        dt_ = t.ones_like(p_y).unsqueeze(1).to(device) * dt
        a_xy = t.zeros_like(p_xy).to(device)
        for o in o_xy:
            d_xy = p_xy - o
            r = t.sqrt(t.sum(d_xy**2, axis=1))
            r_wavelength = r**inner_power * wavelength
            F = G * mass / r**2
            if type_trig == "sin":
                F *= t.sin(r_wavelength - v_y * t_ * moving_wave) ** power
            elif type_trig == "cos":
                F *= t.cos(r_wavelength - v_y * t_ * moving_wave) ** power
            elif type_trig == "tan":
                F *= t.tan(r_wavelength - v_y * t_ * moving_wave) ** power
            # else:
            #     F *= r_wavelength - v_y * t_ * moving_wave
            a_xy += F.unsqueeze(1) * d_xy / t.stack([r, r], axis=1)
        v_xy += a_xy * dt_
        # calculate the velocity of each particle
        v = t.sqrt(t.sum(v_xy**2, axis=1))
        # clip the velocity to the max velocity which is v_y
        v = t.clip(v, 0, v_y)
        # normalize the velocity vector
        v_xy = v_xy / t.stack([v, v], axis=1)
        # multiply the velocity vector by the max velocity
        v_xy *= v_y

        dt_ = (1 / v_xy)[:, 1].unsqueeze(1)
        t.abs_(dt_)
    p_xy += v_xy * dt_
    if round_x > 0:
        p_xy = p_xy.to("cpu").round(decimals=round_x).to(device)
    trail_xy = t.cat((trail_xy, p_xy.unsqueeze(0)), 0)
    hist = t.histc(trail_xy[i][:, 0].to("cpu"), bins=n_bins, min=-x, max=x).to(device)
    dist = t.cat((dist, hist.unsqueeze(0)), 0)
    t_ += dt

print("Creating particle grid")
# trails = trail_xy.reshape(-1, 2).cpu().numpy().round(round_x)
dist = pd.DataFrame(dist.cpu().numpy())
dist = dist.div(dist.max().max())
if round_x > 0:
    dist = dist.round(round_x)

# stop profiling
# pr.disable()
# pr.dump_stats("profile.prof")

# # print stats
# pr.print_stats(sort="time")


# grid = pd.DataFrame(trails, columns=["x", "y"])
# grid_pt = grid[(grid.x > -x) & (grid.x < x)].pivot_table(
#     index="y", columns="x", aggfunc=len, fill_value=0
# )


# if normalize:
#     # Normalizes entire grid
#     grid_pt = grid_pt.div(grid_pt.max().max())

# grid.dropna(inplace=True)
params = dict(
    type_trig=type_trig,
    G=G,
    mass=mass,
    wavelength=wavelength,
    power=power,
    inner_power=inner_power,
    moving_wave=moving_wave,
    edges_slits=edges_slits,
    barrier_points=barrier_points,
    n_particles=n_particles,
    barrier_y=barrier_y,
    dt=dt,
    v_y=v_y,
    x=x,
    y=y,
    round_x=round_x,
    normalize=normalize,
)
if PLOT_3D:
    print("Creating surface plot")
    plot_surface(dist, gravity_nodes_x, o_y, template, params, show=True)

if PLOT_Z_DISTRIBUTION:
    print("Creating heatmap plot")
    plot_heatmap(dist, gravity_nodes_x, o_y, template, params)

  0%|          | 0/3000 [00:00<?, ?it/s]

100%|██████████| 3000/3000 [00:01<00:00, 1942.82it/s]


Creating particle grid
Creating surface plot
